Вывод текста везде (кроме tika) закомменчен для удобства скроллинга

В данном ноутбуке рассмотрены несколько библиотек для извлечения текста из pdf, doc, docx и rtf файлов

Рассмотренные библиотеки: Apache Tika, PyPDF2, PyMuPDF (fitz), pdfminer.six, python-docx, docx2txt и способ чтения без дополнительных библиотек; так же рассмотрен способ конвертации файлов docx->pdf с помощью библиотеки docx2pdf

In [19]:
!pip install ipython-autotime
%load_ext autotime
import warnings
warnings.filterwarnings("ignore")

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 3.36 s (started: 2022-12-30 18:22:00 +03:00)


# Apache Tika
Для работы обязательно нужны установленная Java SE Development Kit 19.0.1 (по требованиям версия 1.6 и выше) и jar-файл (лежит в директории src); ссылка на <a href="https://www.oracle.com/java/technologies/javase/jdk19-archive-downloads.html">java</a>

Перед запуском функции get_document_text необходимо запустить с помощью java jar-файл C:\Users\username\AppData\Local\Temp\tika-server.jar

Команда для запуска в командной строке java -jar C:\Users\username\AppData\Local\Temp\tika-server.jar

После выполнения все должно работать

Принимаемые функцией get_document_text расширения файлов: .doc, .docx, .pdf, .rtf; Но вообще tika способна работать с практически любыми расширениями файлов (медиа, текст и т.д.)

После окончания работы с tika необходимо выключить открытый jar-файл (для этого вроде как достаточно закрыть cmd)

In [20]:
!pip install tika

time: 3.36 s (started: 2022-12-30 18:22:07 +03:00)


In [21]:
import tika
tika.initVM()

time: 0 ns (started: 2022-12-30 18:22:10 +03:00)


In [22]:
from tika import parser

def get_document_text(filepath):
    doc_type = filepath.split('.')[-1]
    if doc_type in ['doc', 'rtf', 'pdf', 'docx']:
        text = parser.from_file(filepath)['content']
    else:
        print(f'File extention must be either pdf, rtf, doc or docx, got {doc_type}')
    return text.strip()

time: 94 ms (started: 2022-12-30 18:22:10 +03:00)


In [23]:
path_doc="C:/Users/265375/Desktop/Work/parsing/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.doc"
print(get_document_text(path_doc))

Образец простого договора аренды квартиры, заключаемого между физическим лицами

ДОГОВОР НАЙМА КВАРТИРЫ
	г. _______________
	«____» ______________ 2022 г.


 Гр. ________________________________________________, паспорт: серия ________, № ________, выданный ________________________, проживающий по адресу: ________________________________________________, именуемый в дальнейшем «Наймодатель», с одной стороны, и гр. ________________________________________________, паспорт: серия ________, № ________, выданный ________________________, проживающий по адресу: ________________________________________________, именуемый в дальнейшем «Наниматель», с другой стороны, именуемые в дальнейшем «Стороны», заключили настоящий договор, в дальнейшем «Договор», о нижеследующем: 
1. Наймодатель предоставляет Нанимателю во временное владение и пользование за плату принадлежащую ему по праву собственности квартиру, кадастровый номер: ________ , находящуюся по адресу: ______________________________________

In [24]:
path_docx="C:/Users/265375/Desktop/Work/parsing/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.docx"
print(get_document_text(path_docx))

Образец простого договора аренды квартиры, заключаемого между физическим лицами

ДОГОВОР НАЙМА КВАРТИРЫ
	г. _______________
	«____» ______________ 2022 г.





 Гр. ________________________________________________, паспорт: серия ________, № ________, выданный ________________________, проживающий по адресу: ________________________________________________, именуемый в дальнейшем «Наймодатель», с одной стороны, и гр. ________________________________________________, паспорт: серия ________, № ________, выданный ________________________, проживающий по адресу: ________________________________________________, именуемый в дальнейшем «Наниматель», с другой стороны, именуемые в дальнейшем «Стороны», заключили настоящий договор, в дальнейшем «Договор», о нижеследующем: 
1. Наймодатель предоставляет Нанимателю во временное владение и пользование за плату принадлежащую ему по праву собственности квартиру, кадастровый номер: ________ , находящуюся по адресу: ___________________________________

In [25]:
path_pdf="C:/Users/265375/Desktop/Work/parsing/Договоры аренды/Договор_аренды_автомобильного_прицепа.pdf"
print(get_document_text(path_pdf))

Образец договора аренды автомобильного прицепа, заключаемого между юридическим и физическим лицом


ДОГОВОР АРЕНДЫ&NBSP;ДОГОВОР АРЕНДЫ&NBSP;
автомобильного прицепаавтомобильного прицепа

г. _______________г. _______________ «_____» _______________ 2022 г.«_____» _______________ 2022 г.

________________________________________________ в лице________________________________________________ в лице
________________________________________________, действующего на основании________________________________________________, действующего на основании
________________________________________________, именуемый в дальнейшем «________________________________________________, именуемый в дальнейшем «АрендодательАрендодатель», с», с
одной стороны, и гр. ________________________________________________, паспорт: серия ________, №одной стороны, и гр. ________________________________________________, паспорт: серия ________, №
________, выданный ________________________, проживающий по адресу:_______

## **Библиотека PyPDF2.**
Работает только с PDF-файлами

In [26]:
!pip install pypdf2
from PyPDF2 import PdfFileReader

time: 3.44 s (started: 2022-12-30 18:22:14 +03:00)


Время чтения и вывода pdf-файла размером 11 страниц - 10.8 с, но при этом шрифт с тенью выводится в единственном экземпляре (+)

In [27]:
pdf_pypdf2 = 'C:/Users/265375/Desktop/Work/parsing/Договоры аренды/Договор_аренды_автомобильного_прицепа.pdf'

with open(pdf_pypdf2, "rb") as filehandle:  
    pdf = PdfFileReader(filehandle)
   
    info = pdf.getDocumentInfo()
    pages = pdf.getNumPages()

    for i in range(pages):
        page = pdf.getPage(i)
        print(page.extractText())

time: 31 ms (started: 2022-12-30 18:22:17 +03:00)


## **PyMuPDF (он же fitz)**
Работает с форматами **PDF**, XPS, OpenXPS, CBZ, EPUB и FB2
<p>Чувствителен к теням у шрифта

In [28]:
!pip install pymupdf
import fitz

time: 3.39 s (started: 2022-12-30 18:22:17 +03:00)


Время чтения и вывода pdf-файла размером 11 страниц - 53 мс

In [29]:
pdf_fitz = "C:/Users/265375/Desktop/Work/parsing/Договоры аренды/Договор_аренды_автомобильного_прицепа.pdf"
text = ''
with fitz.open(pdf_fitz) as doc:
    for page in doc:
        text += page.get_text()
#Из-за тени в исходном документе происходит дублирование, данная строка выделяет одну из двух
text = '\n'.join(text.split('\n')[::2])
print(text)

time: 15 ms (started: 2022-12-30 18:22:21 +03:00)


В fitz можно раскрашивать конкретные слова

In [30]:
text = ''
highlight_text = ['Договор','Квартир']
doc = fitz.open(pdf_fitz)
for current_page in doc:
    for word in highlight_text:
        text_instances = current_page.search_for(word)
        for inst in text_instances:
            highlight = current_page.add_highlight_annot(inst)
            highlight.set_colors({"stroke":(0, 0.5, 0)})
            highlight.update()
doc.save("C:/Users/265375/Desktop/Work/parsing/Договор_аренды_автомобильного_прицепа_colored.pdf", garbage=4, deflate=True, clean=True)

time: 157 ms (started: 2022-12-30 18:22:21 +03:00)


## pdfminer.six (версия pdfminer для python 3.*)
Работа только с pdf-файлами
<p> Чувствителен к теням

In [31]:
!pip install pdfminer.six
from pdfminer.high_level import extract_text

time: 3.5 s (started: 2022-12-30 18:22:21 +03:00)


Время чтения и вывода pdf-файла размером 11 страниц - 2.18 с

In [32]:
pdf_pdfminer = 'C:/Users/265375/Desktop/Work/parsing/Договоры аренды/Договор_аренды_автомобильного_прицепа.pdf'
text_pdfminer = extract_text(pdf_pdfminer)
#Из-за тени в исходном документе происходит дублирование, данная строка выделяет одну из двух
text_pdfminer = '\n'.join(text_pdfminer.split('\n')[::2])
print(text_pdfminer)

The PDF <_io.BufferedReader name='C:/Users/265375/Desktop/Work/parsing/Договоры аренды/Договор_аренды_автомобильного_прицепа.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


time: 1 s (started: 2022-12-30 18:22:24 +03:00)


## **Чтение docx файлов**

## **python-docx (docx)**
Работа только с docx

In [33]:
!pip install python-docx
import docx

time: 4.22 s (started: 2022-12-30 18:22:25 +03:00)


Не смог разобраться, почему не работает

In [34]:
doc = 'C:/Users/265375/Desktop/Work/parsing/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.docx'
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)
getText(doc)

XMLSyntaxError: Start tag expected, '<' not found, line 1, column 1 (<string>, line 1)

time: 218 ms (started: 2022-12-30 18:22:29 +03:00)


## **docx2txt**
Работает только с форматом docx простым переводом в txt

In [37]:
!pip install docx2txt 
import docx2txt

time: 3.41 s (started: 2022-12-30 18:22:46 +03:00)


Сохраняет форматирование, время чтения и вывода - 18 мс

In [38]:
text = docx2txt.process("C:/Users/265375/Desktop/Work/parsing/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.docx")
print(text)

time: 109 ms (started: 2022-12-30 18:22:50 +03:00)


## **Простой метод, использующий стандартную библиотеку zipfile**
Для работы с docx файлами

In [39]:
try:
    from xml.etree.cElementTree import XML
except ImportError:
    from xml.etree.ElementTree import XML
import zipfile

time: 0 ns (started: 2022-12-30 18:22:52 +03:00)


Основан на python-docx, не требует дополнительной установки библиотек, простая функция для чтения, время чтения и вывода - 11.7 мс

In [40]:
WORD_NAMESPACE = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'
PARA = WORD_NAMESPACE + 'p'
TEXT = WORD_NAMESPACE + 't'


def get_docx_text(path):
    """
    Take the path of a docx file as argument, return the text in unicode.
    """
    document = zipfile.ZipFile(path)
    xml_content = document.read('word/document.xml')
    document.close()
    tree = XML(xml_content)

    paragraphs = []
    for paragraph in tree.iter(PARA):
        texts = [node.text
                 for node in paragraph.iter(TEXT)
                 if node.text]
        if texts:
            paragraphs.append(''.join(texts))

    return '\n\n'.join(paragraphs)

time: 15 ms (started: 2022-12-30 18:22:56 +03:00)


In [41]:
docx = 'C:/Users/265375/Desktop/Work/parsing/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.docx'
print(get_docx_text(docx))

time: 0 ns (started: 2022-12-30 18:23:02 +03:00)


## Aspose-words
Платная библиотека (в бесплатной версии остается водяной знак и ограничен функционал)
<p> Работает с doc, doc, odt, pdf, html </p>
<p> Чувствителен к теням

In [44]:
!pip install aspose-words
import aspose.words as aw


time: 3.56 s (started: 2022-12-30 18:23:18 +03:00)


In [45]:
path1="C:/Users/265375/Desktop/Work/parsing/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.doc"

doc = aw.Document(path1)
saveOptions = aw.saving.PdfSaveOptions()
saveOptions.compliance = aw.saving.PdfCompliance.PDF17 
doc.save("C:/Users/265375/Desktop/Work/parsing/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.pdf", saveOptions)
pdf_fitz = "C:/Users/265375/Desktop/Work/parsing/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.pdf"

doc = fitz.open(pdf_fitz)
text = ''
for current_page in range(doc.page_count):
    text += doc.get_page_text(current_page,"text")
#Из-за тени в исходном документе происходит дублирование, данная строка выделяет одну из двух
text = '\n'.join(text.split('\n')[::2])
print(text)

time: 9.33 s (started: 2022-12-30 18:23:21 +03:00)


## Конвертер docx2pdf
Хорошо, но достаточно долго конвертирует docx и doc файлы в pdf

In [46]:
!pip install docx2pdf
from docx2pdf import convert

time: 4.22 s (started: 2022-12-30 18:23:31 +03:00)


In [47]:
convert("C:/Users/265375/Desktop/Work/parsing/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.docx", "C:/Users/265375/Desktop/Work/parsing/test/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.pdf")

  0%|          | 0/1 [00:00<?, ?it/s]

time: 22 s (started: 2022-12-30 18:23:39 +03:00)
